### Finding the true anomaly at a given time
This notebook describes how to find the true anomaly of a small body on an orbit around the Earth at a given time after perigee passage. The procedure shown here finds a solution to Kepler's equation by Newton's method. This illustrates how a numerical solution can be found to an equation that has no analytic solution.
References are made to the book *Spacecraft Systems Engineering (4th Edition)* by Fortescue, Swinerd and Stark, (hereafter SSE).
We begin by importing the packages that we will need.

In [1]:
from math import * # Note, in passing, that math includes the value of pi

We now need to define the parameters required for the calculation. Some of these are physical parameters relating to the orbit, but we also need a parameter (tol - for 'tolerance') that determines the accuracy required for the numerical solution for the eccentric anomaly.

In [3]:
# Physical parameters
mu = 3.986E+14 # Value for Earth in units of m^3 s^-2",
a = 2.0E+07    # Semi-major axis of orbit in m",
e = 0.50       # Eccentricity of orbit",
# Numerical parameter",
tol = 1.0E-05  # Newton's method stops when successive solutions differ by less than this amount

We can now calculate the orbital period (tau) and mean motion (n)

In [4]:
n = sqrt(mu/(pow(a,3)))
tau = 2.*pi/n
n, tau

(0.00022321514285549715, 28148.56208589367)

Note that these quantities are expressed in SI units (n in s^-1 and tau in s)
We are attempting to calculate the true anomaly at one given time, which we will call t1. Remember that time t=0 is the time of perigee.

In [5]:
t1 = 2751.6 # Time in s

Now we calculate the mean anomaly M"

In [6]:
M = n*t1
M

0.6141987870811859

The heart of the calculation is the expression that is used iteratively to obtain estimates of the eccentric anomaly *E* (SSE Eqn 4.20). Because we will be repeatedly evaluating this expression, it is useful to define a function that returns the value of this expression. We call the function Enewton to remind ourselves of the method that is being applied.

In [7]:
def Enewton(Eold,M,e):
    return (Eold-((Eold-e*sin(Eold)-M))/(1.-e*cos(Eold)))

We start the numerical solution by setting the first value of *E* (called Ei) to *M* and then evaluating the next value.

In [8]:
Ei = M
Enext = Enewton(Ei,M,e)
diff=fabs(Enext-Ei)
print(Ei,Enext,diff)

0.6141987870811859 1.10145039465642 0.4872516075752341


Note that we have also calcuated the difference between our starting value, where we set the eccentric anomaly to M, and the value obtained after the first iteration. We need to test whether the difference exceeds the tolerance we specified, and until it does, we iterate until succesive solutions differ by less than this amount.

We do this using a **while** loop in Python. As the program loops through, we print out input and output values of E and the difference between them.

In [9]:
while (diff>tol):
    Ei=Enext
    Enext=Enewton(Ei,M,e)
    diff=fabs(Enext-Ei)
    print(Ei,Enext,diff)
Efinal=Enext
Efinal

1.10145039465642 1.0480556400089003 0.05339475464751975
1.0480556400089003 1.047216338215682 0.0008393017932182278
1.047216338215682 1.0472161347993252 2.034163568698233e-07


1.0472161347993252

This is the value of the eccentric anomaly that we are seeking. We now need to find the true anomaly *theta*, using SSE Eqn 4.14. Looking at this equation, we have to be careful in evaluating the tan() function, since it has an undefined value when its argument is pi/2, 3pi/2 etc. In fact, because E is always in the range 0-2pi, we only need to trap the case where *E*/2=pi/2, i.e *E*=pi. From SSE Figure 4.6 you should see that if *E*=pi, then *theta*=pi.

In [10]:
if (fabs(Efinal-pi/2.)<tol):
    theta=pi
else:
    theta=2.*atan((tan(Efinal/2.))*(sqrt(((1+e))/(1-e))))                  
theta

1.5708177851758547

This result is in radians. To finish off we output this in degrees and provide the starting information.

In [11]:
print(' An orbit around the Earth with a={0:.4e} m, e={1:.3f}'.format(a,e))
print(' Has an orbital period of {0:.1f} s.'.format(tau))
print(' At a time {0:.1f} s after perigee, the true anomaly is {1:.1f} degrees'.format(t1,degrees(theta)))

 An orbit around the Earth with a=2.0000e+07 m, e=0.500
 Has an orbital period of 28148.6 s.
 At a time 2751.6 s after perigee, the true anomaly is 90.0 degrees
